# Project 4 – Numerical Integration
## MATH 3316 High Performance Scientific Computing, Fall 2016

Author: Paul Herz

The designated programming language for this project was C++, built in Xcode and compiled with Clang (Apple LLVM 8.0.0) using the C++14 dialect.

### Structure of this Project

This project's file structure follows the GNU-style C++ project standard.

In [1]:
%ls ..

HPSCProject4.xcodeproj/ lib/                    src/
bin/                    notebooks/
data/                   reports/


Excluding the `.xcodeproj` file, which was used for integration with the Xcode IDE, debugging, and profiling purposes, each item in the project directory serves the following purpose:

- `Makefile`: GNU Make project build automation definitions
- `bin/`: compiled binaries. Make will put binaries here by default.
- `data/`: where calculated data is stored after program execution. Files are `.txt` files containing real numbers, space delimited to denote row items, and newline delimited to denote new rows.
- `lib/`: reused libraries that are not part of this project specifically. Contains my rewrite of the Matrix library ([phrz/matrix](https://github.com/phrz/matrix)).
- `notebooks/`: Python 3 Jupyter notebooks, notably the one used to generate this report.
- `reports/`: PDFs generated from the Jupyter notebooks via `nbconvert` and `pdflatex`.
- `src/`: contains the main mathematical routines described in this report, which are used to generate the data and redirected output in `data/`.

### Using this Project

#### Prerequisites
- A Unix or Unix-like OS (e.g. macOS or Linux)
- A compiler with support for C++14 (LLVM or GNU toolchain)
- Python ≥3.5
- The latest Jupyter distribution
- LaTeX with `pdflatex`
- GNU Make ≥3.81

#### Building this project
This project provides several GNU Make targets, with a handful of them being especially useful.

`make all` (default) - will compile binaries, execute them to generate data files, execute Jupyter notebooks under fresh kernels with just-generated data files, and convert them to PDFs in `/reports`.

`make all_bin` - will compile binaries for parts A, B, and C (`test_int`, `test_adapt`, and `test_carbon`) of this project.

`make all_data` - will compile binaries and execute them to generate data files.

`make clean` - will delete all compiled binaries, generated data, executed notebook copies (but not the original notebooks), and **report PDFs**.

Below, find attached the full Makefile:

In [2]:
%cat ../Makefile

#
#  Makefile
#  HPSCProject4
#
#  Created by Paul Herz on 12/1/16.
#  Copyright © 2016 Paul Herz. All rights reserved.
#

TARGETA = test_int
TARGETB = test_adapt
TARGETC = test_carbon

CXX = g++
CFLAGS = -std=c++14

SRC = src/
LIB = lib/
BIN = bin/
ROOT = $(shell pwd)/
DATA = data/
NB = notebooks/
RP = reports/

AFILES = test_int.cpp
BFILES = test_adapt.cpp
CFILES = test_carbon.cpp
CLIBFILES = Vector.cpp Matrix.cpp

NOTEBOOKABC = $(NB)proj4.ipynb
NOTEBOOKC = $(NB)carbon.ipynb
REPORTABC = $(RP)proj4.pdf
REPORTC = $(RP)carbon.pdf



################################
# All target                   #
################################

all: all_bin all_data all_reports



################################
# Application binaries         #
################################

all_bin: $(TARGETA) $(TARGETB) $(TARGETC)

$(TARGETA):
	$(CXX) $(CFLAGS) -o $(BIN)$(TARGETA) $(addprefix $(SRC), $(AFILES))

$(TARGETB):
	$(CXX) $(CFLAGS) -o $(BIN)$(TARGETB) $(addprefix $(SRC), $(BFILES))

$(TARGETC):
	$(CXX

## Part A – Composite Numerical Integration

### Goals

This part is concerned with the development of a composite integration algorithm of sufficiently fast convergence. We were provided base example code that implemented 2-node Gaussian composite integration.

### Math Background

#### Gaussian Quadrature

In the Gaussian Quadrature algorithm for composite integration, one evaluates $n$ subintervals of the interval of integration $[a,b]$ by dividing it into intervals of width $h=(b-a)/n$. For each subinterval $s_i$, where $i \in \mathbb{Z}, [0,n-1]$, the algorithm considers the midpoint $m=a+(i+0.5)*h$. From this midpoint, a set of nodes are determined as offset from the midpoint. Corresponding to these nodes are weights $w_j$. In the case of the two-node example code, the offsets are $x_0=-\sqrt{1/3},\ x_1 = \sqrt{1/3}$ and the weights are both $1$.
The $j^{\mathrm{th}}$ node is calculated as $N_j=m+0.5*h*x_j$. Note well that the offsets are scaled based on subinterval size rather than added directly! 

Now, using our set of evaluation points $N_0,N_1$ (in the two-node case),  integrand $f(x)$ and known weights $w_0,w_1$, we evaluate the $i^{\mathrm{th}}$ subinterval:

$$
S(i) = \sum_{j} w_j * f(N_j(i))
$$

Where in the two node case,

$$
N_0(i) = m(i) + 0.5*h*-\sqrt{1/3}
$$

$$
N_1(i) = m(i) + 0.5*h*\sqrt{1/3}
$$

and where $m(i)$ yields the $i^{\mathrm{th}}$ midpoint,
$$
m(i) = a + (i+0.5)*h
$$

Assuming $[a,b]$ and $h$ are variables of the algorithm. Then to calculate the full interval we sum the subintervals:

$$
I = \frac{h}{2} \sum_{i} S(i)
$$

### Implementation Requirements

#### Composite Numerical Integration

I had to implement a composite numerical integration algorithm that could converge on an actual integral value on the order of $h^8$ (not to be confused with [Big-O notation](https://en.wikipedia.org/wiki/Big_O_notation), an upper bound function). The example code given was 2-node Gaussian quadrature which converged on the order of $h^8$, so I attempted to re-implement it as 4-node ($n=3$) Gaussian quadrature, primarily by expanding the array of nodes and weights using formulae found in a table on Wikipedia ([Wikipedia: Gaussian Quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature)).

Initially, this algorithm appeared to work well and converge correctly, but this was due to a grave misunderstanding of the instructions, which used Big-O notation to mean "on the order of," which runs contrary to the use of this notation in Computer Science (although I have found it is used differently in some maths). My algorithm originally converged on the order of $h^2$, but because Big-O notation was used, I assumed that $\mathcal{O}(h^2)\in\mathcal{O}(h^8)$ (which would be correct under the nomenclature of computer science but not of some maths), and that I could move on. 

I was correctly very shortly before the due date, and sought to improve the convergence of my algorithm, which qualified individuals believed should have been on the order of $h^8$. I tried many adjustments to my algorithm, but finally found that by precalculating my weights and nodes and inserting them into my code as decimal literals, I was losing a lot of precision compared to just expressing them as mathematical statements and allowing C++ to handle high-precision calculation. This resolved the wide discrepancy of order of convergence between my $h^2$ and the typical $h^8$ of four node Composite Gaussian Quadrature.

#### Integration Testing

My test file follows the example of `test_Gauss2` and uses the function:
$$
f(x) = e^{cx} + \sin(dx)
$$
whose exact antiderivative is:
$$
F(x) = \frac{e^{cx}}{c} - \frac{\cos{dx}}{d}
$$

I first evaluate the true integral:
$$
I = \int_{a}^{b} f(x)dx = F(b) - F(a)
$$

And then, at each $n\in\lbrace 20, 40, 80, 160, 320, 640, 1280, 2560, 5120, 10240 \rbrace$, I use my `composite_int(f,a,b,n)` function to approximate function `f` over `[a,b]` using `n` subintervals. I record
a table with a row listing `n`, `R` (the approximation), `e` (the relative error) and convergence.

Relative error is calculated using true integral $I$ and approximation $R$:
$$
\epsilon = \frac{|I-R|}{|I|}
$$

And the $i^{\mathrm{th}}$ convergence exponent is calculated:
$$
C_i = \frac{ \log(\epsilon_{i-1}) - \log(\epsilon_i) }{ \log(h_{i-1}) - \log(h_i) }
$$

Where $\epsilon_i$ is the error for the current iteration of $n$ and $\epsilon_{i-1}$ is that of the prior iteration, and the same subscripts of $h$ represent the current and prior iterations as well. As such, the convergence exponent is incalculable for reasons of indexing and common sense (a single datapoint has no delta!)

### Implementation

#### Composite Numerical Integration

In [3]:
%cat ../src/composite_int.cpp

//
//  composite_int.cpp
//  HPSCProject4
//
//  Created by Paul Herz on 11/15/16.
//  Copyright © 2016 Paul Herz. All rights reserved.
//

#ifndef _s
#define _s std::to_string
#endif

#include <iostream>
#include <exception>
#include <cmath>

using Fcn = std::function<double(double)>;

// Integrate function `f` on the interval `a` to `b`
// with `n` subintevals of width |b-a|/n.

// NOTE: using template instead of Fcn directly to avoid the overhead of
// std::function, see:
// http://blog.demofox.org/2015/02/25/avoiding-the-performance-hazzards-of-stdfunction/

template<typename T>
double composite_int(T& f, const double a, const double b, const int n) {
	// implement a composite numerical integration formula that is
	// O(h^8) accurate.
	
	// PRECONDITIONS:
	// (1) [a,b] is a valid, forward interval.
	if(b < a) {
		throw std::invalid_argument("The interval [a,b] is invalid, b < a.");
	}
	
	// (2) `n` is a nonzero quantity (integer)
	if(n < 1) {
		throw std::invalid_argument("The numb

#### Integration Testing

In [5]:
%cat ../src/test_int.cpp

//
//  test_int.cpp
//  HPSCProject4
//
//  Created by Paul Herz on 11/15/16.
//  Copyright © 2016 Paul Herz. All rights reserved.
//

#include <iostream>
#include <cmath>
#include <vector>
#include <experimental/optional>
#include <string>
#include <sstream>

#include "println.cpp"
#include "composite_int.cpp"

template<class T>
using optional = std::experimental::optional<T>;

using namespace std;

string leftPad(string s, size_t size, char pad = ' ') {
	size_t difference = size - s.size();
	return string(difference, pad) + s;
}

string rightPad(string s, size_t size, char pad = ' ') {
	size_t originalSize = s.size();
	if(originalSize == size) {
		return s;
	}
	
	// either truncate or pad the string depending
	// on requested size vs actual size.
	s.resize(size, pad);
	
	// if shrunken, replace last character with
	// ellipsis character.
	if(originalSize > size) {
		s.replace(originalSize-1, 1, "…");
	}
		
	return s;
}

template<typename T>
string leftPad(T t, size_t size, char pad =

#### Output

In [7]:
%cat ../data/test_int_out.txt

True integral: 23.9241
┌────────┬────────────┬─────────────┬───────────────┐
|      n |   approx   |    error    |  convergence  |
├────────┼────────────┼─────────────┼───────────────┤
│     20 │ 23.9249    │ 3.48056e-05 │ n/a           │
│     40 │ 23.9241    │ 1.76071e-07 │ 7.627017      │
│     80 │ 23.9241    │ 2.43717e-10 │ 9.496734      │
│    160 │ 23.9241    │ 7.94769e-13 │ 8.260457      │
│    320 │ 23.9241    │ 2.37599e-15 │ 8.385862      │
│    640 │ 23.9241    │ 1.48499e-15 │ 0.678072      │
│   1280 │ 23.9241    │ 7.42497e-16 │ 1.000000      │
│   2560 │ 23.9241    │ 1.63349e-15 │ -1.137504     │
│   5120 │ 23.9241    │ 5.93998e-16 │ 1.459432      │
│  10240 │ 23.9241    │ 5.93998e-16 │ 0.000000      │
└────────┴────────────┴─────────────┴───────────────┘


Note that I satisfy the convergence requirement (of being on the order of $h^8$) as per the parameters of the section of the project. Once the algorithm has converged sufficiently on the value at lower $n$ values, the convergence order fluctuates, as convergence slows down as the approximate value is perhaps already so precise - just a hypothesis.

## Part B – Adaptive Numerical Integration

### Goals

This part is concerned with building upon the composite integration algorithm in the first part. Whereas the first part would approximate `n` subintervals where `n` was given before returning successfully, this part will target a certain degree of accuracy as the condition of success.

### Implementation Requirements

#### Adaptive Numerical Integration



#### Adaptive Testing


### Implementation

#### Adaptive Numerical Integration